In [27]:
import requests
import pandas as pd

df = pd.DataFrame(columns=["steamid", "num_games_owned", "num_reviews", "playtime_forever", "playtime_at_review", "review", "voted_up", 
                           "votes_up", "votes_funny", "weighted_vote_score", "comment_count", "steam_purchase", "received_for_free",
                           "written_during_early_access"])

* The following are the keys from the JSON objects returned from our Steam get call that we will use and what they represent:
    * 'steamid' : the Steam ID of the reviewer
    * 'num_games_owned' : number of the games the reviewer owns
    * 'num_reviews' : count of total amount of reviews that are positive
    * 'playtime_forever' : total play time of the game in minutes
    * 'playtime_at_review' : total play time of the game when the review was written in minutes
    * 'review' : text review of the game the review provided
    * 'voted_up' : did this review recieve votes up from other reviewer (boolean)
    * 'votes_up' : the number of upvotes this review received
    * 'votes_funny': the number of people find this review funny
    * 'weighted_vote_score' : helpfulness score generated by steam on a scale of 0 to 1
    * 'comment_count' : the number of comments this review received from other users
    * 'steam_purchase' : did the user purchase the game on steam (bool)
    * 'received_for_free' : did the user receive this game for free (bool)
    * 'written_during_early_access' : was the review written during early access stage of the game (bool) 

In [28]:
gameID = "275850"
cursor = ''

#pulling all of the reviews, since each call can only pull 20 reviews, we will run this 100 times and stop when it runs out
for i in range(100):

    requestURL = "https://store.steampowered.com/appreviews/" + gameID + "?json=1"+ "&language=english&"+str(cursor)
    response_reviews = requests.get(requestURL).json()
    #each pull generates a cursor and it direct us to the next page
    if 'cursor' in response_reviews:
        cursor = 'cursor='+response_reviews['cursor']
    else:
        break
    for review in response_reviews["reviews"]:
        ''''reviewer's data'''
        gamer = review['author']
        steamid = gamer['steamid']
        num_games_owned = gamer['num_games_owned']
        num_reviews = gamer['num_reviews']
        playtime_forever = gamer ['playtime_forever']
        playtime_at_review = gamer ['playtime_at_review']

        '''actual review data'''
        data = review['review']
        if review['voted_up'] is True:
            voted_up = 1
            votes_up = review['votes_up']
        else:
            voted_up = 0
            votes_up = 0
        votes_funny = review['votes_funny']
        weighted_vote_score = review['weighted_vote_score']
        comment_count = review['comment_count']
        steam_purchase = review['steam_purchase']
        received_for_free = review ['received_for_free']
        written_during_early_access = review['written_during_early_access']

        info = {
            "steamid":steamid, "num_games_owned":num_games_owned, "num_reviews":num_reviews, "playtime_forever":playtime_forever,
            "playtime_at_review" : playtime_at_review, "review" : data, "voted_up": voted_up, "votes_up":votes_up,
            "votes_funny" : votes_funny, "weighted_vote_score" : weighted_vote_score, "comment_count" : comment_count, 
            "steam_purchase" : steam_purchase, "received_for_free" : received_for_free, 
            "written_during_early_access" : written_during_early_access
            }
        df = df.append(info, ignore_index=True)
        


In [29]:
df

,steamid,num_games_owned,num_reviews,playtime_forever,playtime_at_review,review,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access
0,76561198392543189,54,2,7345,7335,"During the disastrous launch of No Man’s Sky, ...",1,589,4,0.950451076030731201,0,True,False,False
1,76561198057055742,117,15,92,88,"To be honest, It's not my style of game.\nBut ...",1,141,1,0.909961581230163574,0,True,False,False
2,76561198107346352,96,8,3703,3703,I don't think anybody would deny that Hello Ga...,0,0,13,0.736209750175476074,37,True,False,False
3,76561198202701980,146,2,8890,8890,This game has more long-term support than most...,1,38,1,0.710257232189178467,0,True,False,False
4,76561198079365268,93,3,13758,9750,i found a giant weenis monster\ni rode the gia...,1,30,34,0.649442493915557861,0,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,76561198080739218,964,2,6072,6042,"THE PROS\n1. the 1st time experience, the < 10...",0,0,0,0.518828451633453369,0,True,False,False
531,76561198045785427,286,5,7250,7114,This game is only as good or bad as you make it.,1,1,0,0.518518567085266113,0,True,False,False
532,76561198084100039,37,3,2200,266,This game is great if you want a space explora...,1,1,0,0.518518567085266113,0,True,False,False
533,76561198035655467,230,34,3952,3952,No other game has been getting so many BIG gam...,1,1,0,0.518072307109832764,0,True,False,False


In [31]:
#export the data
df.to_csv("nms_reviews.csv")